In [6]:
import pandas as pd
import string, re
from pathlib import Path
from collections import Counter

In [7]:
inputdir = Path("../data/resilience/merged")
df = pd.read_csv(Path(inputdir, "merged_wos_files.csv"), encoding = 'utf-8', index_col=[0])
df.head()

,PT,AU,BA,BE,GP,AF,BF,CA,TI,SO,...,WC,WE,SC,GA,PM,OA,HC,HP,DA,UT
0,J,"Jiang, YW; Ritchie, BW; Verreynne, ML",NaN,NaN,NaN,"Jiang, Yawei; Ritchie, Brent W.; Verreynne, Ma...",NaN,NaN,Developing disaster resilience: A processual a...,TOURISM MANAGEMENT,...,"Environmental Studies; Hospitality, Leisure, S...",Social Science Citation Index (SSCI),Environmental Sciences & Ecology; Social Scien...,WQ3JE,NaN,NaN,NaN,NaN,2023-08-03,WOS:000713714900002
1,J,"Nystrom, M; Graham, NAJ; Lokrantz, J; Norstrom...",NaN,NaN,NaN,"Nystrom, M.; Graham, N. A. J.; Lokrantz, J.; N...",NaN,NaN,Capturing the cornerstones of coral reef resil...,CORAL REEFS,...,Marine & Freshwater Biology,Science Citation Index Expanded (SCI-EXPANDED),Marine & Freshwater Biology,368JC,NaN,NaN,NaN,NaN,2023-08-03,WOS:000260616400011
2,J,"Cassidy, S",NaN,NaN,NaN,"Cassidy, Simon",NaN,NaN,The Academic Resilience Scale (ARS-30): A New ...,FRONTIERS IN PSYCHOLOGY,...,"Psychology, Multidisciplinary",Social Science Citation Index (SSCI),Psychology,EC3JI,27917137.0,"Green Published, gold",NaN,NaN,2023-08-03,WOS:000388022000001
3,J,"Adeyeye, K; Emmitt, S",NaN,NaN,NaN,"Adeyeye, Kemi; Emmitt, Stephen",NaN,NaN,"Multi-scale, integrated strategies for urban f...",INTERNATIONAL JOURNAL OF DISASTER RESILIENCE I...,...,Environmental Studies,Emerging Sources Citation Index (ESCI),Environmental Sciences & Ecology,FW7HT,NaN,NaN,NaN,NaN,2023-08-03,WOS:000425493200003
4,J,"Wiig, S; Haraldseid-Driftland, C; Dombestein, ...",NaN,NaN,NaN,"Wiig, Siri; Haraldseid-Driftland, Cecilie; Dom...",NaN,NaN,Backstage researching resilience researchers -...,INTERNATIONAL JOURNAL OF HEALTH GOVERNANCE,...,Health Policy & Services,Emerging Sources Citation Index (ESCI),Health Care Sciences & Services,D3XV6,NaN,hybrid,NaN,NaN,2023-08-03,WOS:000904828800001


In [8]:
df['WC']

0      Environmental Studies; Hospitality, Leisure, S...
1                            Marine & Freshwater Biology
2                          Psychology, Multidisciplinary
3                                  Environmental Studies
4                               Health Policy & Services
                             ...                        
495                Computer Science, Information Systems
496                         Business, Finance; Economics
497    Environmental Studies; Social Sciences, Interd...
498    Engineering, Industrial; Operations Research &...
499                                   Engineering, Civil
Name: WC, Length: 3000, dtype: object

In [9]:
df.loc[:, "WC"] = df.loc[:, "WC"].fillna("")
df['wos_categs'] = df['WC'].str.split(';')
df['wos_categs']

0      [Environmental Studies,  Hospitality, Leisure,...
1                          [Marine & Freshwater Biology]
2                        [Psychology, Multidisciplinary]
3                                [Environmental Studies]
4                             [Health Policy & Services]
                             ...                        
495              [Computer Science, Information Systems]
496                      [Business, Finance,  Economics]
497    [Environmental Studies,  Social Sciences, Inte...
498    [Engineering, Industrial,  Operations Research...
499                                 [Engineering, Civil]
Name: wos_categs, Length: 3000, dtype: object

In [10]:
print(df.shape)
df_wc_explode = df.explode("wos_categs")
print(df_wc_explode.shape)
df_wc_explode.head(10)

(3000, 72)
(5544, 72)


,PT,AU,BA,BE,GP,AF,BF,CA,TI,SO,...,WE,SC,GA,PM,OA,HC,HP,DA,UT,wos_categs
0,J,"Jiang, YW; Ritchie, BW; Verreynne, ML",NaN,NaN,NaN,"Jiang, Yawei; Ritchie, Brent W.; Verreynne, Ma...",NaN,NaN,Developing disaster resilience: A processual a...,TOURISM MANAGEMENT,...,Social Science Citation Index (SSCI),Environmental Sciences & Ecology; Social Scien...,WQ3JE,NaN,NaN,NaN,NaN,2023-08-03,WOS:000713714900002,Environmental Studies
0,J,"Jiang, YW; Ritchie, BW; Verreynne, ML",NaN,NaN,NaN,"Jiang, Yawei; Ritchie, Brent W.; Verreynne, Ma...",NaN,NaN,Developing disaster resilience: A processual a...,TOURISM MANAGEMENT,...,Social Science Citation Index (SSCI),Environmental Sciences & Ecology; Social Scien...,WQ3JE,NaN,NaN,NaN,NaN,2023-08-03,WOS:000713714900002,"Hospitality, Leisure, Sport & Tourism"
0,J,"Jiang, YW; Ritchie, BW; Verreynne, ML",NaN,NaN,NaN,"Jiang, Yawei; Ritchie, Brent W.; Verreynne, Ma...",NaN,NaN,Developing disaster resilience: A processual a...,TOURISM MANAGEMENT,...,Social Science Citation Index (SSCI),Environmental Sciences & Ecology; Social Scien...,WQ3JE,NaN,NaN,NaN,NaN,2023-08-03,WOS:000713714900002,Management
1,J,"Nystrom, M; Graham, NAJ; Lokrantz, J; Norstrom...",NaN,NaN,NaN,"Nystrom, M.; Graham, N. A. J.; Lokrantz, J.; N...",NaN,NaN,Capturing the cornerstones of coral reef resil...,CORAL REEFS,...,Science Citation Index Expanded (SCI-EXPANDED),Marine & Freshwater Biology,368JC,NaN,NaN,NaN,NaN,2023-08-03,WOS:000260616400011,Marine & Freshwater Biology
2,J,"Cassidy, S",NaN,NaN,NaN,"Cassidy, Simon",NaN,NaN,The Academic Resilience Scale (ARS-30): A New ...,FRONTIERS IN PSYCHOLOGY,...,Social Science Citation Index (SSCI),Psychology,EC3JI,27917137.0,"Green Published, gold",NaN,NaN,2023-08-03,WOS:000388022000001,"Psychology, Multidisciplinary"
3,J,"Adeyeye, K; Emmitt, S",NaN,NaN,NaN,"Adeyeye, Kemi; Emmitt, Stephen",NaN,NaN,"Multi-scale, integrated strategies for urban f...",INTERNATIONAL JOURNAL OF DISASTER RESILIENCE I...,...,Emerging Sources Citation Index (ESCI),Environmental Sciences & Ecology,FW7HT,NaN,NaN,NaN,NaN,2023-08-03,WOS:000425493200003,Environmental Studies
4,J,"Wiig, S; Haraldseid-Driftland, C; Dombestein, ...",NaN,NaN,NaN,"Wiig, Siri; Haraldseid-Driftland, Cecilie; Dom...",NaN,NaN,Backstage researching resilience researchers -...,INTERNATIONAL JOURNAL OF HEALTH GOVERNANCE,...,Emerging Sources Citation Index (ESCI),Health Care Sciences & Services,D3XV6,NaN,hybrid,NaN,NaN,2023-08-03,WOS:000904828800001,Health Policy & Services
5,J,"Malindi, MJ; Theron, LC",NaN,NaN,NaN,"Malindi, Macalane J.; Theron, Linda C.",NaN,NaN,The hidden resilience of street youth,SOUTH AFRICAN JOURNAL OF PSYCHOLOGY,...,Social Science Citation Index (SSCI),Psychology,648SO,NaN,NaN,NaN,NaN,2023-08-03,WOS:000281714500010,"Psychology, Multidisciplinary"
6,J,"Kennedy, S; Linnenluecke, MK",NaN,NaN,NaN,"Kennedy, Steve; Linnenluecke, Martina K.",NaN,NaN,Circular economy and resilience: A research ag...,BUSINESS STRATEGY AND THE ENVIRONMENT,...,Social Science Citation Index (SSCI),Business & Economics; Environmental Sciences &...,4I3IW,NaN,"hybrid, Green Published",NaN,NaN,2023-08-03,WOS:000747677200001,Business
6,J,"Kennedy, S; Linnenluecke, MK",NaN,NaN,NaN,"Kennedy, Steve; Linnenluecke, Martina K.",NaN,NaN,Circular economy and resilience: A research ag...,BUSINESS STRATEGY AND THE ENVIRONMENT,...,Social Science Citation Index (SSCI),Business & Economics; Environmental Sciences &...,4I3IW,NaN,"hybrid, Green Published",NaN,NaN,2023-08-03,WOS:000747677200001,Environmental Studies


In [11]:
# strip leading / trailing punctuation
def strip_outer_punct(txt: str):
    newtxt = re.sub(r"^[\W\s]*(.*?)[\W\s]*$", r"\1", txt)
    if newtxt != txt:
        print(newtxt)
    return(newtxt)
df_wc_explode['wos_categs'] = df_wc_explode['wos_categs'].apply(strip_outer_punct)


Hospitality, Leisure, Sport & Tourism
Management
Environmental Studies
Management
Economics
Geography
Management
Water Resources
Economics
Environmental Sciences
Environmental Studies
Computer Science, Theory & Methods
Engineering, Electrical & Electronic
Mathematics, Applied
Environmental Sciences
Environmental Studies
Engineering, Electrical & Electronic
Environmental Sciences
Environmental Studies
Green & Sustainable Science & Technology
Energy & Fuels
Regional & Urban Planning
Urban Studies
Meteorology & Atmospheric Sciences
Water Resources
Rehabilitation
Sociology
Economics
Health Policy & Services
Physics, Applied
Psychology, Multidisciplinary
Engineering, Electrical & Electronic
Optics
Water Resources
Environmental Studies
Geography
Geography, Physical
Regional & Urban Planning
Urban Studies
Engineering, Mechanical
Transportation
Transportation Science & Technology
Psychiatry
Psychology
Transportation Science & Technology
Green & Sustainable Science & Technology
Engineering, Env

In [12]:
# import WoS dataframe exploded by WC category
# return
## df + groups
## dr grouped by groups
## df grouped by groups and year

In [13]:
wos_groups = pd.read_csv(Path("../data/JCR_CategoriesResults_groups.csv"), encoding = 'utf-8', index_col=[0])
wos_groups["Category"] = wos_groups["Category"].str.lower()
wos_groups = wos_groups.rename(columns={"# of journals": "journal_ct", "Citable Items": "citableitems_ct", "Total Citations": "totalcits_ct"})
wos_groups_sub = wos_groups.loc[:, ["Category", "Group"]]

In [14]:
df = df_wc_explode.copy()

In [16]:
df["wos_categs"] = df["wos_categs"].fillna("")
df["wos_categs"] = df["wos_categs"].str.lower()
df2 = pd.merge(df, wos_groups_sub, how="inner", left_on="wos_categs", right_on="Category")

In [18]:
df2["Group"] = df2["Group"].fillna("")
df2["Group"] = df2["Group"].str.split(";")
df2["Group"] = df2["Group"].apply(lambda x: [item.strip() for item in x])
df2_explode = df2.explode("Group")

In [27]:
wos_groups = pd.read_csv(Path("../data/JCR_CategoriesResults_groups.csv"), encoding = 'utf-8', index_col=[0])
wos_groups["Category"] = wos_groups["Category"].str.lower()
wos_groups = wos_groups.rename(columns={"# of journals": "journal_ct", "Citable Items": "citableitems_ct", "Total Citations": "totalcits_ct"})
wos_groups_sub = wos_groups.loc[:, ["Category", "Group"]]

# strip leading / trailing punctuation
def strip_outer_punct(txt: str):
    newtxt = re.sub(r"^[\W\s]*(.*?)[\W\s]*$", r"\1", txt)
    #if newtxt != txt:
        #print(newtxt)
    return(newtxt)

def wos_add_and_explode_groups(df: pd.DataFrame):
    """
    Reads in a WoS dataframe (created from a csv file with
        two-letter abbreviations like "PT" and "AU")
    returns a dataframe exploded by both WoS categories ("WC") 
        and groups (imported and matched with categ's separately)
    """
    print(f"original dataset has {df.shape[0]} rows and {df.shape[1]} columns")
    #Clean df and explode by "WC" categories
    df.loc[:, "WC"] = df.loc[:, "WC"].fillna("")    
    df['wos_categs'] = df['WC'].str.split(';')
    df_wc_explode = df.explode("wos_categs")
    df_wc_explode['wos_categs'] = df_wc_explode['wos_categs'].apply(strip_outer_punct)
    df_wc_explode["wos_categs"] = df_wc_explode["wos_categs"].fillna("")
    df_wc_explode["wos_categs"] = df_wc_explode["wos_categs"].str.lower()
    print(f"exploding by wos_categ (WC) produces a df with {df_wc_explode.shape[0]} rows and {df_wc_explode.shape[1]} columns")

    #add in WoS group data
    df2 = pd.merge(df_wc_explode, wos_groups_sub, how="inner", left_on="wos_categs", right_on="Category")

    # explode by groups
    df2["Group"] = df2["Group"].fillna("")
    df2["Group"] = df2["Group"].str.split(";")
    df2["Group"] = df2["Group"].apply(lambda x: [item.strip() for item in x])
    df2_explode = df2.explode("Group")

    # need to remove duplicate group categories 
    df2_explode = df2_explode.drop_duplicates(subset = ["UT", "Group"])

    print(f"after adding in WoS groups and also exploding by groups, the resulting df has {df2_explode.shape[0]} rows and {df2_explode.shape[1]} columns")

    return df2_explode

In [20]:
inputdir = Path("../data/resilience/merged")
df = pd.read_csv(Path(inputdir, "merged_wos_files.csv"), encoding = 'utf-8', index_col=[0])

In [29]:
newdf = wos_add_and_explode_groups(df)
newdf.head()

original dataset has 3000 rows and 72 columns
exploding by wos_categ (WC) produces a df with 5544 rows and 72 columns
after adding in WoS groups and also exploding by groups, the resulting df has 8571 rows and 74 columns


,PT,AU,BA,BE,GP,AF,BF,CA,TI,SO,...,GA,PM,OA,HC,HP,DA,UT,wos_categs,Category,Group
0,J,"Jiang, YW; Ritchie, BW; Verreynne, ML",NaN,NaN,NaN,"Jiang, Yawei; Ritchie, Brent W.; Verreynne, Ma...",NaN,NaN,Developing disaster resilience: A processual a...,TOURISM MANAGEMENT,...,WQ3JE,NaN,NaN,NaN,NaN,2023-08-03,WOS:000713714900002,environmental studies,environmental studies,Economics & Business
0,J,"Jiang, YW; Ritchie, BW; Verreynne, ML",NaN,NaN,NaN,"Jiang, Yawei; Ritchie, Brent W.; Verreynne, Ma...",NaN,NaN,Developing disaster resilience: A processual a...,TOURISM MANAGEMENT,...,WQ3JE,NaN,NaN,NaN,NaN,2023-08-03,WOS:000713714900002,environmental studies,environmental studies,Multidisciplinary
0,J,"Jiang, YW; Ritchie, BW; Verreynne, ML",NaN,NaN,NaN,"Jiang, Yawei; Ritchie, Brent W.; Verreynne, Ma...",NaN,NaN,Developing disaster resilience: A processual a...,TOURISM MANAGEMENT,...,WQ3JE,NaN,NaN,NaN,NaN,2023-08-03,WOS:000713714900002,environmental studies,environmental studies,"Social Sciences, General"
3,J,"Nystrom, M; Graham, NAJ; Lokrantz, J; Norstrom...",NaN,NaN,NaN,"Nystrom, M.; Graham, N. A. J.; Lokrantz, J.; N...",NaN,NaN,Capturing the cornerstones of coral reef resil...,CORAL REEFS,...,368JC,NaN,NaN,NaN,NaN,2023-08-03,WOS:000260616400011,marine & freshwater biology,marine & freshwater biology,Biology & Biochemistry
3,J,"Nystrom, M; Graham, NAJ; Lokrantz, J; Norstrom...",NaN,NaN,NaN,"Nystrom, M.; Graham, N. A. J.; Lokrantz, J.; N...",NaN,NaN,Capturing the cornerstones of coral reef resil...,CORAL REEFS,...,368JC,NaN,NaN,NaN,NaN,2023-08-03,WOS:000260616400011,marine & freshwater biology,marine & freshwater biology,Environment/Ecology
